# Immobiliare Connector Test

This notebook demonstrates the usage of the immobiliare connector for scraping real estate data.

In [ ]:
import sys
import logging
from datetime import datetime
from pathlib import Path

# Find the project root
project_root = Path().cwd().parent
print(f"Project root: {project_root}")

# Add project root to Python path (not just sources)
sys.path.insert(0, str(project_root))
print(f"Added to Python path: {project_root}")

# Now import from sources
from sources.config import ConfigManager
from sources.connectors.immobiliare import ImmobiliareConnector
from sources.connectors.exceptions import ScrapingError, StorageError

from sources.logging.logging import setup_logging

In [ ]:
# Create a debug configuration
debug_config = {
    'level': 'INFO',  # Set root level to DEBUG
    'handlers': {
        'console': {
            'enabled': True,
            'level': 'INFO'  # Set console to DEBUG
        },
        'file': {
            'enabled': False,
            'level': 'INFO',  # Set file to DEBUG
            'filename': f'immobiliare_scraper_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log',
            'date_format': '%Y%m%d_%H%M%S',
            'directory': 'logs'
        }
    }
}

# Set up logging with debug configuration
setup_logging(debug_config)

# Get the root logger and set its level to DEBUG
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)

# Set specific loggers to DEBUG
logging.getLogger('sources.connectors.immobiliare.scraper').setLevel(logging.INFO)
logging.getLogger('sources.connectors.immobiliare.storage').setLevel(logging.INFO)

# Get the root logger
logger = logging.getLogger(__name__)

In [ ]:
try:
    # Initialize configuration
    logger.info("Initializing configuration...")
    config_manager = ConfigManager()
    
    # Create connector
    logger.info("Creating immobiliare connector...")
    connector = ImmobiliareConnector(config_manager=config_manager)
    
    # Define target URLs
    urls = [
        "https://www.immobiliare.it/vendita-case/milano/",
        #"https://www.immobiliare.it/vendita-case/milano/?criterio=prezzo&ordine=desc"
        #"https://www.immobiliare.it/vendita-case/roma/",
        #"https://www.immobiliare.it/vendita-case/torino/"
    ]
    
    # Scrape each URL
    for url in urls:
        try:
            logger.info(f"Starting scraping from: {url}")
            connector.scrape_and_store(url, max_pages=20)  # Limit to 2 pages per city for testing
            logger.info(f"Successfully scraped data from {url}")
        except ScrapingError as e:
            logger.error(f"Scraping error for {url}: {e}")
        except StorageError as e:
            logger.error(f"Storage error for {url}: {e}")
        except Exception as e:
            logger.error(f"Unexpected error for {url}: {e}")
    
    logger.info("Scraping completed!")
    
except Exception as e:
    logger.error(f"Fatal error: {e}")